In [1]:
model_name = "VietAI/envit5-translation"
# inputs: english caption path
english_caption_path = "/k/AI_Data/CC12M/cc12m_en.tsv"
# outputs: vietnamese caption save path
vietnamese_caption_path = "/k/AI_Data/CC12M/cc12m_vn.tsv"

In [2]:
import csv

from tqdm import tqdm
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to('cuda')

vietnamese_caption_f = open(vietnamese_caption_path, 'w', newline='', encoding="utf-8")
field_name = ['url',  'caption']
writer = csv.DictWriter(vietnamese_caption_f, fieldnames=field_name, delimiter='\t')
writer.writeheader()

# Chọn batch_size nha: [1, 64]
batch_size = 48

count = 0
batch_count = 0
inputs = []
urls = []
with open(english_caption_path, 'r', encoding="utf-8") as tsvfile:
    reader = csv.DictReader(tsvfile, delimiter='\t')
    for row in tqdm(reader):
        try:
            caption = row['caption']
            url = row['url']
            inputs.append(f"en: {caption}")
            urls.append(url)
            batch_count += 1
            if batch_count % batch_size == 0:
                batch_count = 0
                outputs = model.generate(tokenizer(inputs, return_tensors="pt", padding=True).input_ids.to('cuda'), max_length=512)
                vietnamese_batch = tokenizer.batch_decode(outputs, skip_special_tokens=True)
                for vietnamese, url_ in zip(vietnamese_batch, urls):
                    vietnamese = " ".join(vietnamese.split(' ')[0:]).strip()
                    writer.writerow({'url': url_, 'caption': vietnamese})
                    count += 1

                inputs = []
                urls = []
                
            if count % 25 == 0:
                vietnamese_caption_f.flush()
                count = 1
        except Exception as ex:
            print(ex)
            pass

vietnamese_caption_f.close()      

/home/anhcoder/repos/github.com/khoa-nguyendang/simple-stable-diffusion-model/env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-06-20 09:47:51.652317: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-20 09:47:52.429894: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
3119it [05:31,  9.40it/s]


KeyboardInterrupt: 